In [81]:
import oracledb, os, time

In [82]:
# running oracle-free image

def create_connection():
    try:
        connection = oracledb.connect(
            user="SYSTEM",
            password=os.getenv("ORACLE_PASSWORD"),
            dsn="localhost:1523/FREE"          
        )
        print("Verbindung zur PDB erfolgreich!")
    except oracledb.DatabaseError as e:
        print(f"Verbindung fehlgeschlagen: {e}")
    return connection

# Queries

In [83]:
time_stamps_query = """
SELECT TS FROM IOT_TELEMETRY_DATA
"""

In [84]:
#-- 1) Durchschnittliche Luftfeuchtigkeit (humidity) pro Gerät
average_humidity_per_device_query = """SELECT
  DEVICE,
  AVG(HUMIDITY) AS avg_humidity
FROM IOT_TELEMETRY_DATA
GROUP BY DEVICE
ORDER BY DEVICE
"""

In [85]:
# -- 2) Höchste gemessene Temperatur über alle Geräte
highest_temp_query = """SELECT
  MAX(temp) AS max_temperature
FROM IOT_TELEMETRY_DATA
"""

In [86]:
# -- 3) “Worst air quality” per Gerät als Summe der drei Schadstoffe
worst_air_quality_per_device_query = """SELECT
  device,
  SUM(co + lpg + smoke) AS air_quality_score
FROM IOT_TELEMETRY_DATA
GROUP BY device
ORDER BY air_quality_score DESC
"""

In [87]:
avg_temp_humidity_ratio_query = """
SELECT
    AVG(TEMP / NULLIF(HUMIDITY, 0)) AS avg_temp_humidity_ratio
FROM
    IOT_TELEMETRY_DATA
WHERE
    TS >= TO_TIMESTAMP('2020-07-14 00:01:00', 'YYYY-MM-DD HH24:MI:SS')
    AND TS <= TO_TIMESTAMP('2020-07-19 03:01:00', 'YYYY-MM-DD HH24:MI:SS')
"""

In [88]:
import pandas as pd
try:
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute(time_stamps_query)
    time_stamps = pd.DataFrame(cursor.fetchall())
    print(time_stamps.min())
    print(time_stamps.max())
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
0   2020-07-12 00:01:34
dtype: datetime64[ns]
0   2020-07-20 00:03:37
dtype: datetime64[ns]


# Performance Analysis

In [89]:
query_times = {}

In [90]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(average_humidity_per_device_query)
    avg_humidity = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(avg_humidity)
    query_times['Average Humidity'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.11021041870117188 seconds.
[('00:0f:00:70:91:0a', 75.44436079832734), ('1c:bf:ce:15:ec:4d', 61.91024663855242), ('b8:27:eb:bf:9d:51', 50.814076745389464)]


In [91]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(highest_temp_query)
    highest_temp = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(highest_temp)
    query_times['Highest Temp'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)


Verbindung zur PDB erfolgreich!
Successfull fetched in 0.046793222427368164 seconds.
[(30.600000381469727,)]


In [92]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(worst_air_quality_per_device_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Worst Air'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.10737466812133789 seconds.
[('b8:27:eb:bf:9d:51', 6777.162111500038), ('1c:bf:ce:15:ec:4d', 3054.864279412305), ('00:0f:00:70:91:0a', 2785.2340317640083)]


In [93]:

try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(avg_temp_humidity_ratio_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Avg Temp Humidity Ratio'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.14356541633605957 seconds.
[(0.39024171766859717,)]


# Summary

In [94]:
performance = "OracleDB query times: " + str(query_times)
print(performance)

OracleDB query times: {'Average Humidity': 0.11021041870117188, 'Highest Temp': 0.046793222427368164, 'Worst Air': 0.10737466812133789, 'Avg Temp Humidity Ratio': 0.14356541633605957}


In [95]:
filename = "performance.txt"

with open(filename, "a", encoding="utf-8") as f:
    f.write(performance + "\n")

print(performance)


OracleDB query times: {'Average Humidity': 0.11021041870117188, 'Highest Temp': 0.046793222427368164, 'Worst Air': 0.10737466812133789, 'Avg Temp Humidity Ratio': 0.14356541633605957}
